# Run Video Segments Through Qwen2-VL

In [ ]:
!pip install git+https://github.com/huggingface/transformers@21fac7abba2a37fae86106f87fcf9974fd1e3830 accelerate

In [ ]:
!pip install qwen-vl-utils

In [ ]:
!conda install -c conda-forge ffmpeg -y # For a Jupyter Notebook on your own GPU

In [ ]:
import os
import zipfile
import gc
from IPython.display import HTML
from base64 import b64encode

In [ ]:
import torchvision
import transformers
import torch
import os
from math import ceil

print(transformers.__version__)

In [ ]:
!pip install transformers

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype=torch.float32, device_map="auto"
)

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

In [ ]:
def query_video(prompt, frames_path=None, video_path=None):
    messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "video",
                        "video": video_path,
                        "max_pixels": 360 * 420,
                        "fps": 0.1,
                    },
                    {"type": "text", "text": prompt},
                ],
            }
        ]
    image_inputs, video_inputs = process_vision_info(messages)
    
    # Preparation for inference
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    
    inputs = inputs.to("cuda")
    # Inference
    with torch.no_grad():  # Use no_grad to save memory during inference
        # generated_ids = model.encode(**inputs, max_new_tokens=128) generate
        generated_ids = model.generate(**inputs, max_new_tokens=128)
        outputs = model(**inputs, output_hidden_states=True)
        last_hidden_state = outputs.hidden_states[-1] # Get the last layer's hidden states
        embeddings = last_hidden_state[:, -1, :]
        # print(embeddings)
        
    # Trim the generated output to remove the input prompt
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]

    # Decode the generated text
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    print(output_text)
    torch.cuda.empty_cache()

    return embeddings

In [ ]:
import os

directory_path = "task3clips"  # Replace with your directory
for filename in os.listdir(directory_path):
    full_path = os.path.join(directory_path, filename)
    if os.path.isfile(full_path):
        print(f"File found: {full_path}")
        # Perform operations on the file here

In [ ]:
all_embeddings = []

import os

directory_path = "task3clips"  # Replace with your directory
for filename in os.listdir(directory_path):
    full_path = os.path.join(directory_path, filename)
    if os.path.isfile(full_path):
        print(f"File found: {full_path}")

        video_path = full_path
        print(video_path)
        emb = query_video("what are the two mice doing in this video", video_path=video_path)
        all_embeddings.append(emb)
        print(i, emb)
    
        gc.collect()
        torch.cuda.empty_cache()
    
        if i % 10 == 0:
            print("--------------------------------- WE MADE IT TO", i, "---------------------------------")
            torch.save(all_embeddings, 'train_task2_embeddings.pt')
torch.save(all_embeddings, 'train_task2_embeddings.pt')

In [ ]:
torch.save(all_embeddings, 'test_task1_embeddings.pt')

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# obj1 = torch.load('embeddings_ex.pt')
# obj1